In [1]:
%load_ext autoreload

In [2]:
import sys
sys.path.append('../')

In [3]:
import os
import h2o
import pandas as pd

from h2o.automl import H2OAutoML
from utilities import save_model as save

home_dir = os.path.join(os.path.expanduser('~'), 'Documents/TDT4173-ML')
train_data_path = os.path.join(home_dir, 'data_feature_engineering')
target_data_path = os.path.join(home_dir, 'data_processed')

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 48 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_stinky_l3b4cy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,924 Mb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [5]:
train = pd.read_csv(os.path.join(train_data_path, 'mean_fwiz_features_train.csv'))
target = pd.read_csv(os.path.join(target_data_path, 'target.csv'))

train.set_index('date_forecast', inplace=True)
target.set_index('time', inplace=True)

In [6]:
X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
autoML = H2OAutoML(max_models=30, seed=8)
autoML.train(x=X, y=y, training_frame=X_train)

AutoML progress: |


15:53:11.610: XGBoost_1_AutoML_3_20231102_155311 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_3_20231102_155311_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


███████
15:54:02.203: XGBoost_2_AutoML_3_20231102_155311 [XGBoost def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_2_AutoML_3_20231102_155311_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


██████
15:56:26.879: XGBoost_3_AutoML_3_20231102_155311 [XGBoost def_3] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_3_AutoML_3_20231102_155311_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


███████████████

key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/30
# GBM base models (used / total),4/13
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),0/14
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [8]:
best_model = autoML.leader
save.save_model_to_local_disk(best_model, '02E_h2o_mean_fwiz_features')